# Benchmark Piscis Runtime

In [ ]:
!pip install -U piscis==0.2.5 orbax-checkpoint==0.4.4 tensorflow==2.15.0 tensorstore==0.1.45 tf-keras==2.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.4/248.4 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.1/224.1

In [ ]:
import os
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'

import jax
import numpy as np

from jax import random
from piscis import Piscis
from piscis.data import load_datasets
from piscis.downloads import download_dataset
from piscis.utils import pad_and_stack

In [ ]:
# Download the Piscis dataset.
download_dataset('20230905', '')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


(…)ld_Seq0000_crop1to1407_14Jan2019_CY5.npz:   0%|          | 0.00/2.15M [00:00<?, ?B/s]

(…)API,CY3,A594,Brightfield_Seq0001_CY3.npz:   0%|          | 0.00/4.32M [00:00<?, ?B/s]

(…)ts/wniu/Piscis/20230905/A3_GFP1_A594.npz:   0%|          | 0.00/9.13M [00:00<?, ?B/s]

(…)ets/wniu/Piscis/20230905/A3_GFP1_CY3.npz:   0%|          | 0.00/3.55M [00:00<?, ?B/s]

...

(…)wniu/Piscis/20230905/particle_subset.npz:   0%|          | 0.00/15.8M [00:00<?, ?B/s]

(…)wniu/Piscis/20230905/receptor_subset.npz:   0%|          | 0.00/4.04M [00:00<?, ?B/s]

(…)iu/Piscis/20230905/untreated014_A594.npz:   0%|          | 0.00/3.44M [00:00<?, ?B/s]

(…)niu/Piscis/20230905/untreated014_CY5.npz:   0%|          | 0.00/11.7M [00:00<?, ?B/s]

(…)/wniu/Piscis/20230905/vesicle_subset.npz:   0%|          | 0.00/4.06M [00:00<?, ?B/s]

In [ ]:
# Load the Piscis dataset.
ds = load_datasets('20230905', load_train=True, load_valid=True, load_test=True)
images = pad_and_stack(np.concatenate((ds['train']['images'], ds['valid']['images'], ds['test']['images'])))

In [ ]:
# Randomly choose 100 images.
key = random.PRNGKey(0)
images = random.choice(key, images, shape=(100, ))

In [ ]:
# Load the Piscis model.
model = Piscis(model_name='20230905')

wniu/Piscis/models/20230905:   0%|          | 0.00/29.9M [00:00<?, ?B/s]

In [ ]:
# Get CPU and GPU devices.
gpu_device = jax.devices('gpu')[0]
cpu_device = jax.devices('cpu')[0]

In [ ]:
# Benchmark Piscis on CPU.
with jax.default_device(cpu_device):
  cpu_result = %timeit -o model.predict(images, threshold=1.0)
cpu_per_image = cpu_result.average / len(images)
print('CPU Per Image:', cpu_per_image, 's')

5min 13s ± 1.16 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
CPU Per Image: 3.138401407594286 s


In [ ]:
# Benchmark Piscis on GPU.
with jax.default_device(gpu_device):
  gpu_result = %timeit -o model.predict(images, threshold=1.0)
gpu_per_image = gpu_result.average / len(images)
print('GPU Per Image:', gpu_per_image, 's')

4.17 s ± 176 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
GPU Per Image: 0.04170909454571431 s


In [ ]:
import psutil

# Get CPU information.
with open('/proc/cpuinfo') as f:
    for line in f:
        if 'model name' in line:
            cpu_name = line.split(':')[1].strip()
            break
cpu_count = psutil.cpu_count()
cpu_frequency = psutil.cpu_freq().current

# Get RAM information.
ram = round(psutil.virtual_memory().total / (1024 ** 3), 2)

# Get GPU information.
gpu_info = None
try:
    import subprocess
    result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'],
                            stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    gpu_name, gpu_ram = result.stdout.strip().split(', ')
except FileNotFoundError:
    gpu_name = None
    gpu_ram = None

# Print system information.
print('CPU Name:', cpu_name)
print('CPU Count:', cpu_count)
print('CPU Frequency:', cpu_frequency, 'GHz')
print('RAM:', ram, 'GB')
print('GPU Name:', gpu_name)
print('GPU RAM:', gpu_ram, 'GB')

CPU Name: Intel(R) Xeon(R) CPU @ 2.00GHz
CPU Count: 2
CPU Frequency: 2000.144 GHz
RAM: 12.67 GB
GPU Name: Tesla T4
GPU RAM: 15360 MiB GB
